## n_ary function

In [2]:
# ---------------
# User Instructions
#
# Write a function, n_ary(f), that takes a binary function (a function
# that takes 2 inputs) as input and returns an n_ary function. 

def n_ary(f):
    """Given binary function f(x, y), return an n_ary function such
    that f(x, y, z) = f(x, f(y,z)), etc. Also allow f(x) = x."""
    def n_ary_f(x, *args):
        # your code here
        if len(args) > 1:
            return f(x, n_ary_f(args[0], *args[1:]))
        elif len(args) == 1:
            return f(x, args[0])
        else:
            return f(x)
    return n_ary_f
    
def foo(x, y=None):
    return x * y if y is not None else x
    
nary = n_ary(foo)

def test():
    assert nary(3) == 3
    assert nary(1, 2) == 2
    assert nary(1, 2, 3) == 6
    assert nary(1, 2, 3, 4) == 24
    print 'test pass'

test()

test pass


### A better one

In [1]:
def n_ary(f):
    """Given binary function f(x, y), return an n_ary function such
    that f(x, y, z) = f(x, f(y,z)), etc. Also allow f(x) = x."""
    def n_ary_f(x, *args):
        # your code here
        return x if not args else f(x, n_ary_f(*args))
    return n_ary_f
    
def foo(x, y=None):
    print x, y
    return x * y if y is not None else x
    
nary = n_ary(foo)

def test():
    assert nary(3) == 3
    assert nary(1, 2) == 2
    assert nary(1, 2, 3) == 6
    assert nary(1, 2, 3, 4) == 24
    print 'test pass'

test()

3 4
2 12
1 24
test pass


## Update Wrapper

### Decorator could make debug hard because of function name changed

In [6]:
def n_ary(f):
    """Given binary function f(x, y), return an n_ary function such
    that f(x, y, z) = f(x, f(y,z)), etc. Also allow f(x) = x."""
    def n_ary_f(x, *args):
        # your code here
        return x if not args else f(x, n_ary_f(*args))
    return n_ary_f

@n_ary
def foo(x, y=None):
    return x * y if y is not None else x
    
help(foo)

# Output:
# Help on function n_ary_f in module __main__:
# n_ary_f(x, *args)


Help on function n_ary_f in module __main__:

n_ary_f(x, *args)



### Use update_wrapper to fix

In [7]:
from functools import update_wrapper

def n_ary(f):
    """Given binary function f(x, y), return an n_ary function such
    that f(x, y, z) = f(x, f(y,z)), etc. Also allow f(x) = x."""
    def n_ary_f(x, *args):
        # your code here
        return x if not args else f(x, n_ary_f(*args))
    update_wrapper(n_ary_f, f)
    return n_ary_f

@n_ary
def foo(x, y=None):
    return x * y if y is not None else x
    
help(foo)  # Output: foo(x, *args)

Help on function foo in module __main__:

foo(x, *args)



## Decorated Decorators

![DECORATOR](../img/decorated_explain.png)

#### `decorator` updates 2 functions: `d(fn)` and `_d`

In [24]:
# Update a wrapper function to look like the wrapped function. 

from functools import update_wrapper

def decorator(d):
    "Make function d a decorator: d wraps a function fn."
    def _d(fn):
        return update_wrapper(d(fn), fn)
    update_wrapper(_d, d)
    return _d

@decorator
def n_ary(f):
    """Given binary function f(x, y), return an n_ary function such
    that f(x, y, z) = f(x, f(y,z)), etc. Also allow f(x) = x."""
    def n_ary_f(x, *args):
        # your code here
        return x if not args else f(x, n_ary_f(*args))
    return n_ary_f

@n_ary
def foo(x, y=None):
    return x * y if y is not None else x
    
help(foo)  # Output: foo(x, *args)

help(n_ary)

Help on function foo in module __main__:

foo(x, *args)

Help on function n_ary in module __main__:

n_ary(fn)
    Given binary function f(x, y), return an n_ary function such
    that f(x, y, z) = f(x, f(y,z)), etc. Also allow f(x) = x.



### Alternative for decorator

In [19]:
from functools import update_wrapper

def decorator(d):
    return lambda fn: update_wrapper(d(fn), fn)

decorator = decorator(decorator)

@decorator
def n_ary(f):
    """Given binary function f(x, y), return an n_ary function such
    that f(x, y, z) = f(x, f(y,z)), etc. Also allow f(x) = x."""
    def n_ary_f(x, *args):
        # your code here
        return x if not args else f(x, n_ary_f(*args))
    return n_ary_f

@n_ary
def foo(x, y=None):
    return x * y if y is not None else x
    
help(foo)  # Output: foo(x, *args)

help(n_ary)

Help on function foo in module __main__:

foo(x, *args)

Help on function n_ary in module __main__:

n_ary(fn)
    Given binary function f(x, y), return an n_ary function such
    that f(x, y, z) = f(x, f(y,z)), etc. Also allow f(x) = x.



## Unhashable

TypeError: unhashable type: 'list'

In [26]:
d = {}
x = 42
x in d

y = [1, 2, 3]
y in d  # TypeError: unhashable type: 'list'

TypeError: unhashable type: 'list'

List are mutable, not hashable.

Python does not allow you to put list into hashable. Since you could change an item value, and make the list un-traceable.

## Cache Mangement using decorator

In [22]:
@decorator
def memo(f):
    """Cache the return value for each call to f(args)."""
    cache = {}
    def _f(*args):
        try:
            return cache[args]
        except KeyError:
            cache[args] = result = f(*args)
            return result
        except TypeError:
            # some element of args can't be a dict key.
            return f(args)
    return _f

## Count calls using decorator

In [25]:
from functools import update_wrapper

def decorator(d):
    return lambda fn: update_wrapper(d(fn), fn)

decorator = decorator(decorator)

@decorator
def memo(f):
    """Cache the return value for each call to f(args)."""
    cache = {}
    def _f(*args):
        try:
            return cache[args]
        except KeyError:
            cache[args] = result = f(*args)
            return result
        except TypeError:
            # some element of args can't be a dict key.
            return f(args)
    return _f

@decorator
def countcalls(f):
    def _f(*args):
        callcounts[_f] += 1
        return f(*args)
    callcounts[_f] = 0
    return _f
    
callcounts = {}

@countcalls
@memo
def fib(n): return 1 if n <= 1 else fib(n-1) + fib(n-2)

print fib(32)

3524578


# Type of tools



Debug: count calls, trace, disabled

Performance: memo

Expression: n_ary

In [40]:
def disabled(f): return f

trace = disabled

# after this, the trace decorator will just return f itself.

## Impl trace

In [27]:
# ---------------
# User Instructions
#
# Modify the function, trace, so that when it is used
# as a decorator it gives a trace as shown in the previous
# video. You can test your function by applying the decorator
# to the provided fibonnaci function.
#
# Note: Running this in the browser's IDE will not display
# the indentations.

from functools import update_wrapper


def decorator(d):
    "Make function d a decorator: d wraps a function fn."
    def _d(fn):
        return update_wrapper(d(fn), fn)
    update_wrapper(_d, d)
    return _d

@decorator
def trace(f):
    indent = '   '
    def _f(*args):
        signature = '%s(%s)' % (f.__name__, ', '.join(map(repr, args)))
        print '%s--> %s' % (trace.level*indent, signature)
        trace.level += 1
        try:
            # your code here
            result = f(*args)
            print '%s<-- %s === %s' % ((trace.level-1)*indent, 
                                      signature, result)
        finally:
            # your code here
            trace.level -= 1
        #print 'result', result
        return result # your code here
    trace.level = 0
    return _f

@trace
def fib(n):
    if n == 0 or n == 1:
        return 1
    else:
        return fib(n-1) + fib(n-2)

fib(6) #running this in the browser's IDE  will not display the indentations!

--> fib(6)
   --> fib(5)
      --> fib(4)
         --> fib(3)
            --> fib(2)
               --> fib(1)
               <-- fib(1) === 1
               --> fib(0)
               <-- fib(0) === 1
            <-- fib(2) === 2
            --> fib(1)
            <-- fib(1) === 1
         <-- fib(3) === 3
         --> fib(2)
            --> fib(1)
            <-- fib(1) === 1
            --> fib(0)
            <-- fib(0) === 1
         <-- fib(2) === 2
      <-- fib(4) === 5
      --> fib(3)
         --> fib(2)
            --> fib(1)
            <-- fib(1) === 1
            --> fib(0)
            <-- fib(0) === 1
         <-- fib(2) === 2
         --> fib(1)
         <-- fib(1) === 1
      <-- fib(3) === 3
   <-- fib(5) === 8
   --> fib(4)
      --> fib(3)
         --> fib(2)
            --> fib(1)
            <-- fib(1) === 1
            --> fib(0)
            <-- fib(0) === 1
         <-- fib(2) === 2
         --> fib(1)
         <-- fib(1) === 1
      <-- fib(3) === 3
      --> fib

13

In [51]:
def f(*args):
    signature = 'func: %s(%s)' % (f.__name__, ', '.join(map(repr, args)))
    print signature

f(1, 2, 3)

# repr returns a string containing a printable representation of an object.
repr(123)

func: f(1, 2, 3)


'123'